# Lab 2: Converting to an Asynchronous Agent Architecture

## Introduction

Welcome to Lab 2! In this lab, we'll transform our synchronous marketing agent from Lab 1 into an asynchronous system using AWS Lambda and Amazon SQS. This architecture will allow our agent to handle tasks more efficiently and reliably, without being constrained by execution time limits.

By the end of this lab, you'll understand how to:
- Package a Strands Agent for AWS Lambda deployment
- Configure SQS for task management
- Implement an asynchronous execution flow
- Test the asynchronous agent architecture

## Understanding Asynchronous Agents

Before we dive into implementation, let's understand why asynchronous agents are valuable:

### Limitations of Synchronous Agents

Synchronous agents face several constraints:
- **Execution time limits**: Cloud function platforms typically have timeout limits
- **Resource inefficiency**: Compute resources remain allocated during waiting periods
- **Poor scalability**: Difficult to handle concurrent requests efficiently
- **Vulnerability to interruptions**: Long-running processes can fail if connections drop

### Benefits of Asynchronous Architecture

An asynchronous architecture addresses these challenges through:
- **Event-driven execution**: The agent only runs when there's work to do
- **Decoupled components**: Tasks and processing are separated for better resilience
- **Scalable processing**: Multiple agent instances can process tasks in parallel
- **Improved fault tolerance**: Failed tasks can be retried without losing progress

## Architecture Overview

Our asynchronous architecture consists of these key components:

1. **SQS Queue (`post-generator-task-queue`)**: 
   - Stores tasks waiting to be processed
   - Provides at-least-once delivery guarantees
   - Handles message visibility and dead-letter scenarios

2. **Lambda Function (`post_generator_agent`)**:
   - Contains our Strands Agent code
   - Triggered by messages in the SQS queue
   - Processes tasks and publishes results

3. **Task Structure**:
   - JSON payloads containing task details and context

Let's implement this architecture step by step.

⚠️⚠️ Please run the [prerequisites](../lab_0/prerequisites.ipynb) before continuing with this lab, if you haven't done so already. ⚠️⚠️

This lab assumes that:
- An ENVIRONMENT VARIABLE `PUBLISH_API_ENDPOINT` is set with the URL to use to publish to UniTok website.
- A valid `strands` AWS Lambda Layer exists in the AWS account.
- A valid AWS profile exists OR
- Valid AWS credentials are setup.

In [ ]:
%env AWS_DEFAULT_REGION=us-west-2

In [ ]:
# Import required libraries
import boto3

response = boto3.client('cloudformation').describe_stacks(StackName="UniTokStack")
        
# Get the Unito API endpoint URL
for output in response['Stacks'][0]['Outputs']:
    if output['OutputKey'] == "ApiEndpoint":
        PUBLISH_API_ENDPOINT = f'{output["OutputValue"]}posts'
    if output['OutputKey'] == "DistributionDomainName":
        UNITOK_URL = f'http://{output["OutputValue"]}'
    if output['OutputKey'] == "Strandslayer":
        STRANDS_LAYER_ARN = f'{output["OutputValue"]}'
print(f"Using following API endpoint for publishing UniTok posts: {PUBLISH_API_ENDPOINT}")
print(f"You can reach the UniTok site at : {UNITOK_URL}")
print(f"Using following Strands Lambda Layer: {STRANDS_LAYER_ARN}")


## Step 1: Setting Up AWS Resources

First, we need to set up our AWS resources: an SQS queue and a Lambda function.

### Creating the SQS Queue

Let's create the `post-generator-task-queue` SQS queue:

In [ ]:
# Code to create an SQS queue named 'post-generator-task-queue'
import boto3

sqs = boto3.client('sqs')

# Create Dead Letter Queue first
dlq_response = sqs.create_queue(
    QueueName='post-generator-agent-tasks-dead-letter-queue',
    Attributes={
        'VisibilityTimeout': '300',
        'MessageRetentionPeriod': '1209600'  # 14 days
    }
)

# Get the DLQ ARN
dlq_url = dlq_response['QueueUrl']
dlq_attributes = sqs.get_queue_attributes(
    QueueUrl=dlq_url,
    AttributeNames=['QueueArn']
)
dlq_arn = dlq_attributes['Attributes']['QueueArn']

# Create main queue with redrive policy
main_queue_response = sqs.create_queue(
    QueueName='post-generator-agent-tasks',
    Attributes={
        'VisibilityTimeout': '900',
        'MessageRetentionPeriod': '1209600',  # 14 days
        'ReceiveMessageWaitTimeSeconds': '20',
        'RedrivePolicy': f'{{"deadLetterTargetArn":"{dlq_arn}","maxReceiveCount":"3"}}'
    }
)
post_generator_queue_url = main_queue_response['QueueUrl']
post_generator_queue_arn = sqs.get_queue_attributes(
    QueueUrl=post_generator_queue_url,
    AttributeNames=['QueueArn']
)['Attributes']['QueueArn']
print(f"Queue created with URL: {post_generator_queue_url}")

### Creating the Lambda Function

Now, let's prepare our Lambda function:

In [ ]:
# Cell for setting up the Lambda function configuration
import boto3
import os

# Function configuration
function_name = 'PostGeneratorAgent'
handler = 'index.lambda_handler'
runtime = 'python3.11'
architecture = 'arm64'
code_directory = '../functions/post_generator_agent/'
timeout = 900  # Default timeout in seconds
memory_size = 128  # Default memory in MB

# Get resource ARNs needed for policies
sqs_client = boto3.client('sqs')

# Environment variables
environment_variables = {
    'PUBLISH_API_ENDPOINT': PUBLISH_API_ENDPOINT
}

# Create IAM policy document
policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        # Basic Lambda execution
        {
            "Effect": "Allow",
            "Action": [
                "logs:CreateLogGroup",
                "logs:CreateLogStream",
                "logs:PutLogEvents"
            ],
            "Resource": "arn:aws:logs:*:*:*"
        },
        # SQS poll from own queue
        {
            "Effect": "Allow",
            "Action": [
                "sqs:ReceiveMessage",
                "sqs:DeleteMessage",
                "sqs:GetQueueAttributes"
            ],
            "Resource": post_generator_queue_arn
        },
        # Bedrock permissions
        {
            "Effect": "Allow",
            "Action": [
                "bedrock:InvokeModel",
                "bedrock:InvokeModelWithResponseStream"
            ],
            "Resource": "*"
        }
    ]
}

print(f"Function configuration prepared for {function_name}")


### Lambda Handler Function

Now, let's create the Lambda handler function:


In [ ]:
!rm -r ../functions/
!mkdir ../functions
!mkdir ../functions/post_generator_agent/
!echo "Created a new folder to write Lambda functions temporarily: ../functions/"

In [ ]:
%%writefile ../functions/post_generator_agent/index.py
# Lambda function implementation of an async Strands Agents that gets invoked via a task from SQS
import json
import uuid
import boto3
import logging
import os
import requests
from strands import Agent, tool
from strands.models import BedrockModel

logger = logging.getLogger(__name__)

AGENT_NAME = 'post-generator-agent'
PUBLISH_API_ENDPOINT = os.environ.get('PUBLISH_API_ENDPOINT', None)

def prepare(task) -> Agent:
    try:
        type = task.get('type', None)
        assert type is not None, "Task type is not specified"
        assert type in ["new"], "Task type is not supported, must be `new`"
        logger.info(f"Preparing agent for {type} task")
        if type == "new":
            # Structure of a new task
            # {
            #     'type': 'new',
            #     'body': {
            #         'task': 'new task description',
            #     },
            #     'parent': { # Optional
            #         'agent_name': 'name of the agent who requested this task',
            #         'session_id': 'id of the session that the parent is carrying',
            #         'callback_sqs': 'SQS queue url to report the completion of the task',
            #         'tool_use_id': 'id of the tool that was initiaed to call this agent'
            #     }
            # }
            task_body = task.get('body', None)
            assert task_body is not None, "Task body is not specified"
            task_description = task_body.get('task', None)
            assert task_description is not None, "Task description is not specified"


            # Create a new session_id UUID
            session_id = str(uuid.uuid4())
            logger.info(f"New session_id: {session_id}")

            return session_id, task_description

    except Exception as e:
        logger.error(f"Unknown task type: {type}")
        raise e

@tool
def publish_post(
    content: str, 
    author: str = "Unicorn Rentals", 
    unicorn_color: str = "rainbow", 
    image_url: str = None) -> str:
    """
    Publish a post to the UniTok social media platform.

    Args:
        content (str): The text content of the post.
        author (str, optional): The author of the post. Defaults to "Unicorn Rentals".
        unicorn_color (str, optional): The color of the unicorn. Choose from: pink, blue, purple, green, yellow, or rainbow. Defaults to "rainbow".
        image_url (str, optional): URL to an image to include with the post. Defaults to None.

    Returns:
        str: A message indicating the post was published successfully, or an error message.
    """

    # For this lab, we'll simulate posting to UniTok
    print(f"Publishing post to UniTok: {content}")
    print(f"Author: {author}")
    print(f"Unicorn Color: {unicorn_color}")
    if image_url:
        print(f"Image URL: {image_url}")
    post_data = {
        "content": content,
        "author": author,
        "unicornColor": unicorn_color
    }
    if image_url:
        post_data["imageUrl"] = image_url
    try:
        # Send the post to the API
        logger.info(f"Publishing post to UniTok: {post_data}")
        response = requests.post(PUBLISH_API_ENDPOINT, json=post_data)
        logger.info(f"Response from UniTok: {response}")
        # Check if the request was successful
        if response.status_code == 201:
            post_id = response.json().get("postId")
            logger.info(f"Post published successfully! Post ID: {post_id}")
            return f"Post published successfully! Post ID: {post_id}"
        else:
            logger.error(f"Failed to publish post. Status code: {response.status_code}, Response: {response.text}")
            return f"Failed to publish post. Status code: {response.status_code}, Response: {response.text}"
    except Exception as e:
        logger.error(f"Error publishing post with the exception {e}")
        return "Error publishing post with the exception {e}"

def lambda_handler(event, context):
    logger.info(f"Received event: {event}")

    # Even when processing a single message, AWS Lambda still wraps it in a Records array
    if not event.get('Records') or len(event['Records']) == 0:
        logger.error("No records found in the event")
        return {
            'statusCode': 400,
            'body': json.dumps('No SQS message records found in the event')
        }

    # Extract the first (and only) message
    record = json.loads(event['Records'][0]['body'])
    logger.info(f"Processing record: {record}")
    session_id, prompt = prepare(record)
    logger.info(f"Session ID: {session_id}, Prompt: {prompt}")
    INSTRUCTIONS = """
    You are a creative social media manager for Unicorn Rentals, a company that offers unicorns for rent that kids and grown-ups can play with.

    Your task is to create engaging social media posts for UniTok, our unicorn-themed social media platform. 

    Important information about Unicorn Rentals:
    - We offer unicorns in various colors: pink, blue, purple, green, yellow, and rainbow (our most popular)
    - Our new product feature allows customers to pick their favorite color unicorn to rent (default: rainbow)
    - Our target audience includes families with children, fantasy enthusiasts, and event planners
    - Our brand voice is magical, playful, and family-friendly

    When creating posts:
    - Keep content family-friendly and positive
    - Highlight the magical experience of spending time with unicorns
    - Mention the new color selection feature when appropriate
    - Use emojis sparingly but effectively
    - Keep posts between 50-200 characters for optimal engagement

    After generating the post, publish it directly to UniTok using the publish_post tool.
    """
    # Create model
    model = BedrockModel(
        model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0"
    )

    # Create agent
    agent = Agent(
        system_prompt=INSTRUCTIONS,
        model=model,
        tools=[publish_post]
    )

    result = agent(prompt, session_id=session_id)
    logger.info(f"Agent message history {agent.messages}")
    logger.info(str(result))


## Step 2: Packaging and Deployment

Now, let's package our code and deploy it to AWS:


In [ ]:
# Cell for packaging the Lambda function

import zipfile
import tempfile
import os
import shutil

def package_lambda_function(source_dir):
    """
    Package the Lambda function code into a zip file
    """
    if not os.path.exists(source_dir):
        raise FileNotFoundError(f"Source directory {source_dir} not found")
    
    # Create a temporary directory for packaging
    with tempfile.TemporaryDirectory() as temp_dir:
        zip_path = os.path.join(temp_dir, 'function.zip')
        
        with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
            # Walk through all files in the source directory
            for root, _, files in os.walk(source_dir):
                for file in files:
                    file_path = os.path.join(root, file)
                    # Calculate the path within the zip file (relative to source_dir)
                    arcname = os.path.relpath(file_path, source_dir)
                    zipf.write(file_path, arcname)
        
        # Read the zip file content
        with open(zip_path, 'rb') as zip_file:
            zip_content = zip_file.read()
        
        return zip_content

# Package the Lambda function
try:
    lambda_zip_content = package_lambda_function(code_directory)
    zip_size_kb = len(lambda_zip_content) / 1024
    print(f"Lambda function packaged successfully: {zip_size_kb:.2f} KB")
except Exception as e:
    print(f"Error packaging Lambda function: {str(e)}")
    raise


In [ ]:
# Cell for deploying to AWS

import boto3
import json
import time

def create_iam_role(role_name, policy_document):
    """Create an IAM role for the Lambda function"""
    iam_client = boto3.client('iam')
    
    # Create the IAM role with basic Lambda execution trust policy
    trust_policy = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {"Service": "lambda.amazonaws.com"},
                "Action": "sts:AssumeRole"
            }
        ]
    }
    
    try:
        response = iam_client.create_role(
            RoleName=role_name,
            AssumeRolePolicyDocument=json.dumps(trust_policy),
            Description=f"Role for {function_name} Lambda function"
        )
        role_arn = response['Role']['Arn']
        
        # Attach the inline policy to the role
        iam_client.put_role_policy(
            RoleName=role_name,
            PolicyName=f"{role_name}-policy",
            PolicyDocument=json.dumps(policy_document)
        )
        
        # Wait for role to propagate (IAM changes can take time to propagate)
        print("Waiting for IAM role to propagate...")
        time.sleep(10)
        
        return role_arn
    
    except iam_client.exceptions.EntityAlreadyExistsException:
        # If role already exists, get its ARN
        response = iam_client.get_role(RoleName=role_name)
        role_arn = response['Role']['Arn']
        
        # Update the policy
        iam_client.put_role_policy(
            RoleName=role_name,
            PolicyName=f"{role_name}-policy",
            PolicyDocument=json.dumps(policy_document)
        )
        
        return role_arn

def deploy_lambda_function(function_name, handler, runtime, role_arn, zip_content, 
                          layer_arns, environment_vars, architecture, timeout=900, memory_size=128):
    """Deploy the Lambda function to AWS"""
    lambda_client = boto3.client('lambda')
    
    try:
        # Check if function already exists
        try:
            lambda_client.get_function(FunctionName=function_name)
            # Function exists, update it
            print(f"Updating existing Lambda function: {function_name}")
            response = lambda_client.update_function_code(
                FunctionName=function_name,
                ZipFile=zip_content
            )
            
            # Update configuration
            lambda_client.update_function_configuration(
                FunctionName=function_name,
                Runtime=runtime,
                Role=role_arn,
                Handler=handler,
                Layers=layer_arns,
                Timeout=timeout,
                MemorySize=memory_size,
                Environment={
                    'Variables': environment_vars
                },
                Architectures=[architecture]
            )
            
        except lambda_client.exceptions.ResourceNotFoundException:
            # Function doesn't exist, create it
            print(f"Creating new Lambda function: {function_name}")
            response = lambda_client.create_function(
                FunctionName=function_name,
                Runtime=runtime,
                Role=role_arn,
                Handler=handler,
                Code={
                    'ZipFile': zip_content
                },
                Layers=layer_arns,
                Timeout=timeout,
                MemorySize=memory_size,
                Environment={
                    'Variables': environment_vars
                },
                Architectures=[architecture]
            )
        
        return response['FunctionArn']
    
    except Exception as e:
        print(f"Error deploying Lambda function: {str(e)}")
        raise

def create_or_update_event_source_mapping(function_name, queue_arn):
    """Create or update the SQS event source mapping for the Lambda function"""
    lambda_client = boto3.client('lambda')
    
    # List existing event source mappings
    response = lambda_client.list_event_source_mappings(
        FunctionName=function_name,
        EventSourceArn=queue_arn
    )
    
    if response['EventSourceMappings']:
        # Event source mapping exists, update it
        uuid = response['EventSourceMappings'][0]['UUID']
        print(f"Updating existing event source mapping: {uuid}")
        
        lambda_client.update_event_source_mapping(
            UUID=uuid,
            FunctionResponseTypes=['ReportBatchItemFailures'],
            BatchSize=1,
            MaximumBatchingWindowInSeconds=0
        )
    else:
        # Create new event source mapping
        print(f"Creating new event source mapping for queue: {queue_arn}")
        lambda_client.create_event_source_mapping(
            EventSourceArn=queue_arn,
            FunctionName=function_name,
            Enabled=True,
            BatchSize=1,
            MaximumBatchingWindowInSeconds=0,
            FunctionResponseTypes=['ReportBatchItemFailures']
        )

# Deploy the Lambda function
try:
    # Create IAM role for the Lambda function
    role_name = f"{function_name}-role"
    role_arn = create_iam_role(role_name, policy_document)
    print(f"IAM role created/updated: {role_arn}")
    
    # Deploy the Lambda function
    function_arn = deploy_lambda_function(
        function_name=function_name,
        handler=handler,
        runtime=runtime,
        role_arn=role_arn,
        zip_content=lambda_zip_content,
        layer_arns=[STRANDS_LAYER_ARN],
        environment_vars=environment_variables,
        architecture=architecture,
        timeout=timeout,
        memory_size=memory_size
    )
    
    print(f"Lambda function deployed successfully: {function_arn}")
    
    # Create or update the SQS event source mapping
    create_or_update_event_source_mapping(function_name, post_generator_queue_arn)
    print(f"SQS event source mapping created/updated for queue: {post_generator_queue_arn}")
    
    # Get the function details
    lambda_client = boto3.client('lambda')
    function_details = lambda_client.get_function(FunctionName=function_name)
    print(f"Function details: {function_details['Configuration']['FunctionArn']}")
    
except Exception as e:
    print(f"Deployment failed: {str(e)}")
    raise


## Step 4: Testing the Asynchronous Agent

Let's test our asynchronous agent with various scenarios:

### Sending a Task to the Queue

In [ ]:
# Send a marketing task to the SQS queue
task = {
  "type": "new",
  "body": {
    "task": "Create a rental post about playful orange unicorns for the launch of a new fragrance KONFA."
  }
}
sqs = boto3.client('sqs')
sqs.send_message(
    QueueUrl=post_generator_queue_url,
    MessageBody=json.dumps(task)
)

## Viewing Posts on the UniTok Website

Now that we've created and published posts using our agent, let's see them on the UniTok website!


In [ ]:
print(f"UniTok website: {UNITOK_URL}")


## Conclusion

Congratulations! You've successfully transformed your synchronous marketing agent into an asynchronous system using AWS Lambda and SQS. This architecture allows your agent to handle tasks more efficiently and reliably, without being constrained by execution time limits.

Key takeaways from this lab:
- How to structure an asynchronous agent architecture
- How to use SQS for task management
- How to implement a Lambda function that processes agent tasks
- How to test and monitor asynchronous agent execution

In Lab 3, we'll extend this architecture to handle long-running operations with human approval workflows, further enhancing our agent's capabilities.